In [1]:
import numpy as np

# Function Definitions

In [2]:
def py_trapz(f: callable, a: float, b: float, n: int) -> float:
    """
    py_trapz function integrates the function
    f from a to b using n trapeziums
    """
    stepsize = (b - a) / n
    integral = 0
    prev = f(a)  # stores the functions value at thefirst point
    for i in range(1, n + 1):
        end = a + stepsize * i  # end (x_value) of the trapz
        end_val = f(end)  # f(x) at end value
        integral += prev + end_val
        prev = end_val  # current end is the start of the next trap
    return integral * stepsize / 2

In [3]:
%load_ext cython

In [4]:
%%cython --annotate

import cython


@cython.boundscheck(False)
@cython.wraparound(False)
def  cy_trapz(object f, double a,double b,int n):
    """
    cython function that integrates the function f
    from a to b using n trapeziums
    """
    
    cdef double stepsize = (b - a) / n
    cdef int i
    cdef double integral = 0
    cdef double prev = f(a) #initial functional value
    cdef double end, endval
    
    for i in range(1,n+1):
        end = a + stepsize*i #end(x_value) of the trapz
        endval = f(end) #f(x) at end value
        integral += (prev + endval)
        prev = endval #current end is the start of the next trap
        
    return integral * stepsize / 2


In [5]:
def find_values(g: callable, a: float, b: float, n: int, actual_value: float):
    """
    helper function that takes in a function, the limits,
    the number of trapeziums, and the actual value,
    and performs a timing, and accuracy analysis
    for the three methods :
    py_trapz, cy_trapz and np.trapz
    """

    value1 = py_trapz(g, a, b, n)
    value2 = cy_trapz(g, a, b, n)
    x_vals = np.linspace(a, b, n + 1)  # creating x value array
    value3 = np.trapz(g(x_vals), x_vals)

    accuracy = lambda x: 1 - abs(
        (actual_value - x) / actual_value
    )  # lambda function to calculate accuracy
    print(f"py_trapz : Value : {value1}, Accuracy : {accuracy(value1)}")
    print("time :")
    %timeit py_trapz(g, a, b, n)
    print(f"cy_trapz : Value : {value2}, Accuracy : {accuracy(value2)}")
    print("time :")
    %timeit cy_trapz(g, a, b, n)
    print(f"numpy_trapz : Value : {value3}, Accuracy : {accuracy(value3)}")
    print("time :")
    %timeit np.trapz(g(x_vals), x_vals)

# Analysis for 4 cases

a) x^2 from 0 to 1
b)sinx from 0 to pi
c) e^x from 0 to 1
d) 1/x from 1 to 2

number of trapeziums used is 100 in all of the above test cases

In [6]:
print(f"Calculations for x^2 from 0 to 1")
find_values(lambda x: x * x, 0, 1, 100, 1 / 3)

Calculations for x^2 from 0 to 1
py_trapz : Value : 0.3333499999999999, Accuracy : 0.9999500000000001
time :
12.8 µs ± 295 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
cy_trapz : Value : 0.3333499999999999, Accuracy : 0.9999500000000001
time :
5.94 µs ± 96.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
numpy_trapz : Value : 0.33335000000000004, Accuracy : 0.9999499999999999
time :
9.39 µs ± 174 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
print(f"Calculations for sin(x) from 0 to pi")
find_values(np.sin, 0, np.pi, 100, 2)

Calculations for sin(x) from 0 to pi
py_trapz : Value : 1.999835503887444, Accuracy : 0.999917751943722
time :
70.3 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
cy_trapz : Value : 1.999835503887444, Accuracy : 0.999917751943722
time :
57.2 µs ± 770 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
numpy_trapz : Value : 1.9998355038874434, Accuracy : 0.9999177519437217
time :
10.3 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
print(f"Calculations for e(x) from 0 to 1")
find_values(np.exp, 0, 1, 100, np.exp(1) - 1)

Calculations for e(x) from 0 to 1
py_trapz : Value : 1.718296147450418, Accuracy : 0.9999916666805552
time :
64.8 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
cy_trapz : Value : 1.718296147450418, Accuracy : 0.9999916666805552
time :
53.1 µs ± 654 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
numpy_trapz : Value : 1.7182961474504175, Accuracy : 0.9999916666805554
time :
9.85 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [9]:
print(f"Calculations for 1/x from 1 to 2")
find_values(lambda x: 1 / x, 1, 2, 100, np.log(2))

Calculations for 1/x from 1 to 2
py_trapz : Value : 0.6931534304818241, Accuracy : 0.9999909832686995
time :
13.9 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
cy_trapz : Value : 0.6931534304818241, Accuracy : 0.9999909832686995
time :
7.39 µs ± 96.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
numpy_trapz : Value : 0.6931534304818243, Accuracy : 0.9999909832686992
time :
10.6 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Performance Test by integrating x^2 from 0 to 10, using 10,000,000 trapezoids 

In [10]:
print("For python implementation")
%timeit py_trapz(lambda x: x*x, 0, 10, 10000000)

For python implementation
1.4 s ± 9.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
print("For cython implementation")
%timeit cy_trapz(lambda x: x*x, 0, 10, 10000000)

For cython implementation
557 ms ± 3.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
print("For numpy implementation")
x_vals = np.linspace(0, 10, 10000001)
g = lambda x: x * x
%timeit np.trapz(g(x_vals), x_vals)

For numpy implementation
111 ms ± 602 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Alternate Cython Function Implementation

This implementation, has the function to generate values as a cython c function itself, rather than a separate python function, leading to quite a large jump in performance

In [13]:
%%cython --annotate
import cython
#creating the c function that returns x^2
cdef double fx(float x):
    return x*x

@cython.boundscheck(False)
@cython.wraparound(False)
def  cy_trapz_modified(double a,double b,int n):
    """
    cython function that integrates the function f
    from a to b using n trapeziums
    """
    
    cdef double stepsize = (b - a) / n
    cdef int i
    cdef double integral = 0
    cdef double start, end

    for i in range(n):
        start = a + stepsize * i
        end = start + stepsize
        integral += (fx(start) + fx(end)) 
    
    return integral * stepsize / 2

In [14]:
print("For cython implementation")
print(cy_trapz_modified(0, 10, 10000000))
%timeit cy_trapz_modified( 0, 10, 10000000)

For cython implementation
333.33333333571636
23.3 ms ± 362 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
